In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# 输入研究区的shp文件所在路径
wuhan_shp ="D:/NDVI_exercise/wuhan/wuhanregion.shp"
# shp 转 ee 
wuhan_ee = geemap.shp_to_ee(wuhan_shp)
# 添加到layer
# Map.addLayer(wuhan_ee,{},"wuhan")
Map.centerObject(wuhan_ee,7)   # 设置以研究区居中

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# landsat8 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}
# JRC数据集的可视化参数
visualization = {
    'bands': ['waterClass'],
    'min': 0.0,
    'max': 3.0,
    'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
# NDVI and mNDWI 函数的定义
def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'


In [5]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(wuhan_ee).select('constant').rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")
basemap1 = ee.Image.constant(0).clip(wuhan_ee).select('constant').rename('class')

In [ ]:
# JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
# JRC_permanentwater = JRC_wuhan_water.eq(3).selfMask()
# Map.addLayer(JRC_permanentwater,{'palette':['blue']},'jrcpermanent')

# 训练&分类

In [6]:
landsat8_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan_ee) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr)

landsat_ndvi_mndwi = landsat8_images.map(NDVI).map(mNDWI)
# 计算有效像元的个数
validPixel = landsat8_images.count().select('B3').clip(wuhan_ee).rename('count')
# Map.addLayer(validPixel,{},'count_image')

In [7]:
Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(wuhan_ee).select('b1').gt(74).select('b1').rename('class')
wuhan_Maryland = ee.ImageCollection([Maryland_2018,basemap1]).sum()
wuhan_JRC = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee).eq(3).select("waterClass").rename('class')
Maryland_JRC = ee.ImageCollection([wuhan_Maryland,wuhan_JRC]).sum().eq(2)
Maryland_JRC_samplepoints = Maryland_JRC.sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

# 70% 做训练点位，30% 做验证点位
withRandom = Maryland_JRC_samplepoints.randomColumn('random')

#把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
split = 0.7
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

def imageSample(image):
    # Overlay the points on the imagery to get training.
    sample_point = image.select(bands).sampleRegions(**{
        'collection': trainingPartition,
        'properties': ['class'],
        'scale': 30
    })
    return sample_point
points_collection = landsat_ndvi_mndwi.map(imageSample).flatten()
# print('total sample number:{}'.format(points_collection.size().getInfo()))
# print(points_collection.first().getInfo())
# samplepoint_water = points_collection.filter(ee.Filter.eq('jrcwaterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))
# print(samplepoint_water.first().getInfo())

trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,'class',bands)
#对Landsat-8进行分类
def training(image):
    return image.clip(wuhan_ee).select(bands).classify(trainedClassifier).eq(1).selfMask().select('classification').rename('class')
landsatImage_classify = landsat_ndvi_mndwi.map(training)
waterpixel_count = landsatImage_classify.sum()
# print(landsatImage_classify.size().getInfo())
# Map.addLayer(waterpixel_count,{},"waterPixelCount")
wuhan_waterfrequency = waterpixel_count.select('class').divide(validPixel.select('count')).select('class').rename('frequency')
wuhan_permanentwater = wuhan_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
# Map.addLayer(wuhan_waterfrequency,{},"waterfrequency")
# Map.addLayer(wuhan_permanentwater,{'palette':['red']},"permanentwater")



In [10]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
permanent_water_layer = imagecollection.sum()
Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'bands': ['waterclass'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# # use cartoee to get a map
# ax = cartoee.get_map(wuhan_permanentwater, region=region, vis_params=vis)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'permanent surface water in 2018(JRC+Maryland)', fontsize=20)

# 验证

In [9]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

JRC_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': testingPartition,
    'properties': ['class'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('class', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

{'type': 'Feature', 'geometry': None, 'id': '3_0', 'properties': {'class': 0, 'waterclass': 0}}
[[1396, 4], [15, 82]]
total caccuracy:0.9873079492317969
kappa:0.889441708419657


In [ ]:
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
JRC_permanentwater = JRC_wuhan_water.eq(3).selfMask()
Map.addLayer(JRC_permanentwater,{'palette':['blue']},'jrcpermanent')

# 水体面积

In [ ]:
# calculate the area of permanent water
waterarea = wuhan_permanentwater.multiply(ee.Image.pixelArea()).divide(1e6)
areas = waterarea.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': wuhan_ee.geometry(),
    'scale': 30,
    'maxPixels': 1e14
})
print(areas.getInfo())